In [1]:
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
from sentinelhub import CRS, BBox, DataCollection, SHConfig

In [2]:

data_folder="test_dir"

In [3]:
lat = 38.13105
lon = 23.8779

In [4]:
config = SHConfig()
config = SHConfig()
config.sh_client_id = 'sh-dce21bb4-8cfb-48df-8f32-8c655d0a83f1'
config.sh_client_secret = '6ZiLKeRpeyTNfrovxYLVgnMN7qj0HUv1'
config.sh_base_url = 'https://sh.dataspace.copernicus.eu'
config.sh_token_url = 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token'

if config.sh_client_id == "" or config.sh_client_secret == "":
    print("Warning! To use Sentinel Hub Catalog API, please provide the credentials (client ID and client secret).")

In [5]:
from sentinelhub import SentinelHubCatalog

catalog = SentinelHubCatalog(config=config)

#catalog.get_info()

In [6]:
caspian_sea_bbox = BBox((lon-0.01, lat-0.01, lon+0.01, lat+0.01), crs=CRS.WGS84)
time_interval = "2024-07-20", "2024-07-28"

search_iterator = catalog.search(
    DataCollection.SENTINEL2_L2A,
    bbox=caspian_sea_bbox,
    time=time_interval,
    filter="eo:cloud_cover < 5",
    fields={"include": ["id", "properties.datetime", "properties.eo:cloud_cover"], "exclude": []},
)

results = list(search_iterator)
print("Total number of results:", len(results))

results

Total number of results: 4


[{'id': 'S2A_MSIL2A_20240728T090601_N0511_R050_T34SGH_20240728T152859.SAFE',
  'properties': {'datetime': '2024-07-28T09:19:54.009Z',
   'eo:cloud_cover': 1.99}},
 {'id': 'S2A_MSIL2A_20240728T090601_N0511_R050_T35SKC_20240728T152859.SAFE',
  'properties': {'datetime': '2024-07-28T09:19:52.991Z',
   'eo:cloud_cover': 1.66}},
 {'id': 'S2B_MSIL2A_20240723T090559_N0511_R050_T34SGH_20240723T120201.SAFE',
  'properties': {'datetime': '2024-07-23T09:19:54.629Z',
   'eo:cloud_cover': 0.14}},
 {'id': 'S2B_MSIL2A_20240723T090559_N0511_R050_T35SKC_20240723T120201.SAFE',
  'properties': {'datetime': '2024-07-23T09:19:53.609Z',
   'eo:cloud_cover': 0.11}}]

In [7]:
from sentinelhub import MimeType, SentinelHubDownloadClient, SentinelHubRequest, bbox_to_dimensions, filter_times

time_difference = dt.timedelta(hours=1)

all_timestamps = search_iterator.get_timestamps()
unique_acquisitions = filter_times(all_timestamps, time_difference)

unique_acquisitions

[datetime.datetime(2024, 7, 23, 9, 19, 53, 609000, tzinfo=tzutc()),
 datetime.datetime(2024, 7, 28, 9, 19, 52, 991000, tzinfo=tzutc())]

In [9]:
from cgi import test


evalscript_all_bands = """
    //VERSION=3
    function setup() {
        return {
            input: [{
                bands: ["B01","B02","B03","B04","B05","B06","B07","B08","B8A","B09","B10","B11","B12","SCL"],
                units: "DN"
            }],
            output: {
                bands: 13,
                sampleType: "INT16"
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B01,
                sample.B02,
                sample.B03,
                sample.B04,
                sample.B05,
                sample.B06,
                sample.B07,
                sample.B08,
                sample.B8A,
                sample.B09,
                sample.B10,
                sample.B11,
                sample.B12,
                sample.SCL];
    }
"""

request_all_bands = SentinelHubRequest(
    data_folder = data_folder,
    evalscript=evalscript_all_bands,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L1C.define_from("s2l1c", service_url=config.sh_base_url),
            time_interval=("2024-07-20", "2024-07-28"),
                 )
    ],
    responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
    bbox=caspian_sea_bbox,
    size = bbox_to_dimensions(caspian_sea_bbox, resolution=10),
    config=config,
)

In [10]:
all_bands_response = request_all_bands.get_data(save_data=True)


DownloadFailedException: Failed to download from:
https://sh.dataspace.copernicus.eu/api/v1/process
with HTTPError:
400 Client Error: Bad Request for url: https://sh.dataspace.copernicus.eu/api/v1/process
Server response: "{"status": 400, "reason": "Bad Request", "message": "Collection 'S2L1C' has no band 'SCL'.", "code": "RENDERER_EXCEPTION"}"

In [ ]:
%%time

client = SentinelHubDownloadClient(config=config)
download_requests = [request.download_list[0] for request in process_requests]
data = client.download(download_requests)


data[0].shape

CPU times: total: 0 ns
Wall time: 504 ms


(23, 17, 4)

In [ ]:
data

[array([[[ 12,  21,  25, 102],
         [ 19,  30,  34,  74],
         [ 23,  33,  43,  58],
         ...,
         [ 15,  23,  28, 103],
         [ 21,  31,  38,  70],
         [ 18,  27,  36,  72]],
 
        [[ 16,  25,  30,  84],
         [ 14,  23,  23,  56],
         [ 16,  23,  29,  62],
         ...,
         [ 22,  32,  38,  77],
         [ 21,  30,  35,  91],
         [ 20,  30,  41,  79]],
 
        [[ 18,  27,  37,  57],
         [ 15,  24,  30,  58],
         [ 17,  26,  32,  67],
         ...,
         [ 12,  20,  22,  47],
         [ 25,  36,  46,  72],
         [ 16,  29,  43,  88]],
 
        ...,
 
        [[ 24,  34,  46, 105],
         [ 21,  31,  40,  86],
         [ 16,  26,  32,  70],
         ...,
         [ 10,  13,  19,  34],
         [  9,  12,  18,  22],
         [ 10,  12,  19,  13]],
 
        [[ 24,  32,  36,  78],
         [ 19,  28,  31,  75],
         [ 24,  38,  51, 116],
         ...,
         [ 11,  14,  21,  50],
         [ 10,  13,  19,  22],
    